In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset
import random
from litellm import completion, batch_completion
import os
import litellm

In [ ]:
SYSTEM_PROMPT_kor = """
당신의 일은 문제-정답 쌍으로 되어 있는 데이터셋이 주어진 목적에 부합한 지 판단하는 것입니다.
주어진 데이터를 읽고 다음 기준에 얼마나 부합하는 지 1~5점 사이의 점수를 매기세요.

*** 기준 : 
1. 금융학, 투자 전략, 법률, 금융 관련 수학 계산 등 금융 관련 주제에 대한 데이터인지
2. 전문적인 분석과 논리적인 사고를 요구하거나 포함되어있는 데이터인지
3. (문제의 경우) 문제에서 요구하는 내용이 구체적이고 명확하며, 문제 해결에 필요한 정보가 충분한지
4. (정답의 경우) 정답이 정확하고 완전하며, Chain of Thoughts 방식으로 정답을 도출되었는지

*** 점수 기준

5점: 
- 금융 전문 지식이 필요하며 문제와 정답이 매우 구체적이고 완벽함
- 논리적 추론 과정이 명확하게 단계별로 제시됨
- 제시한 기준과 매우 적합함

4점:
- 금융 관련 내용을 포함하며 분석적 사고가 필요함
- 문제와 정답이 구체적이고 추론 과정이 있음
- 제시한 기준과 대부분 적합함

3점:
- 금융과 간접적으로 연관되어 있음
- 기본적인 분석만 필요하고 추론 과정이 불완전하거나 부족함

2점:
- 금융 관련성이 매우 낮고 단순 암기 수준임
- 문제나 정답이 모호하고 추론 과정 없음

1점:
- 금융과 전혀 무관한 주제임
- 문제와 정답이 부적절하거나 불명확함
"""

In [5]:
SYSTEM_PROMPT = """
Your task is to evaluate whether question-answer pairs in a dataset meet the given purpose.
Read the given data and score it on a scale of 1-5 based on how well it meets the following criteria:

*** Evaluation Criteria:

Content Categories:

1. Financial Expertise
- Relevance to financial topics (studies, investment, law, calculations)
- Professional analysis and expertise requirements
- Market efficiency and investor value contribution
2. Logical Structure
- Clear analytical thinking process
- Step-by-step reasoning
- Evidence-based conclusions
3. Practical Value
- Market applicability
- Investor usefulness
- Real-world implementation
4. Format & Compliance
- Natural Korean language usage
- Legal/ethical compliance
- Professional terminology accuracy

*** Scoring Guidelines:

5 points (Outstanding):
- Demonstrates advanced financial expertise
- Perfect step-by-step logical reasoning
- Highly practical and implementable solutions
- Complete alignment with all criteria

4 points (Proficient):
- Shows solid financial knowledge
- Clear logical progression
- Practical application potential
- Meets most evaluation criteria

3 points (Adequate):
- Basic financial relevance
- Partial analytical process
- Limited practical value
- Meets basic requirements

2 points (Limited):
- Minimal financial connection
- Lacks analytical depth
- No practical application
- Major criteria gaps

1 point (Insufficient):
- No financial relevance
- Unclear or inappropriate content
- No analytical value
- Fails to meet basic criteria
"""

In [6]:
import pandas as pd
from datasets import load_dataset, Dataset
import random
from litellm import completion, batch_completion
import os
import litellm

In [ ]:
from pydantic import BaseModel
from typing import List,Literal
import json
class AnswerModel(BaseModel):
    score : Literal["1","2","3","4","5"]
os.environ["OPENAI_API_KEY"] = "sk-XXX"


In [32]:
data_path = 'overfit-brothers/filtered-merged-50k-finqa'

raw = load_dataset(data_path)['train']

Generating train split: 100%|██████████| 35431/35431 [00:00<00:00, 78141.53 examples/s]


In [33]:
raw

Dataset({
    features: ['question', 'answer', 'reference'],
    num_rows: 35431
})

In [34]:
qrys = []
cnt = 0
for t in raw:
    question = t['question']
    answer = t['answer']
    system_prompt = SYSTEM_PROMPT
    prompt = f'Question : {question}\nAnswer : {answer}'
    messages = [
    {"content": system_prompt, "role": "system"},
    {"content": prompt, "role": "user"}]
    qrys.append(messages)
    # cnt+=1
    # if cnt ==100:
    #     break
# 1. raw text 데이터를 활용한 질문 생성
responses = batch_completion(
    model="gpt-4o",
    messages = qrys,
    response_format=AnswerModel,
)
response_resps = [json.loads(i.choices[0].message.content) for i in responses]
total_prompt_tokens_for_q = sum([r.usage.prompt_tokens for r in responses])
total_completion_tokens_for_q = sum([r.usage.completion_tokens for r in responses])

In [35]:
print('total prompt tokens:', total_prompt_tokens_for_q )
print('prompt token costs: $', round((total_prompt_tokens_for_q ) / 1000000 * 2.5, 6))
print('total completion tokens:', total_completion_tokens_for_q )
print('completion token costs: $', round((total_completion_tokens_for_q ) / 1000000 * 10, 6))

total prompt tokens: 34181072
prompt token costs: $ 85.45268
total completion tokens: 177155
completion token costs: $ 1.77155


In [36]:
raw_sample = raw.select(range(100))

In [39]:
sample_list = []
score_counts = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
for score, datum in zip(response_resps,raw):
    score_value = int(score['score'])
    if 1 <= score_value <= 5:
        score_counts[score_value] += 1
    datum['score'] = score_value
    if score_value == 5:
        sample_list.append(datum)

for score in range(1, 6):
    print(f"Score {score}: {score_counts[score]} counts")
    
dataset = Dataset.from_list(sample_list)


Score 1: 47 counts
Score 2: 291 counts
Score 3: 1189 counts
Score 4: 7454 counts
Score 5: 26450 counts


In [41]:
dataset

Dataset({
    features: ['question', 'answer', 'reference', 'score'],
    num_rows: 26450
})

In [42]:
dataset.push_to_hub("overfit-brothers/filtered-merged-50k-finqa_quality_filtered",private=True)

Uploading the dataset shards: 100%|██████████| 1/1 [00:06<00:00,  6.09s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/overfit-brothers/filtered-merged-50k-finqa_quality_filtered/commit/75c87dbed650fe83bde7718ab18b8d7dfc88a3aa', commit_message='Upload dataset', commit_description='', oid='75c87dbed650fe83bde7718ab18b8d7dfc88a3aa', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/overfit-brothers/filtered-merged-50k-finqa_quality_filtered', endpoint='https://huggingface.co', repo_type='dataset', repo_id='overfit-brothers/filtered-merged-50k-finqa_quality_filtered'), pr_revision=None, pr_num=None)

In [38]:
dataset[0]

{'question': '주 6: 재고는 미국 대륙에 위치한 대부분의 재고에 대해 후입선출(lifo) 방식을 사용하며, 그 외의 재고는 후입선출(first-in, first-out) 방식으로 평가된다. 피포 비용은 현재의 대체 비용과 비슷하다. lifo를 이용하여 측정한 재고는 반드시 원가 또는 시장 중에서 낮은 가격으로 평가되어야 한다. 피포를 사용하여 측정한 재고자산은 원가 또는 순실현가능가치 중 낮은 값으로 평가되어야 한다. 12월 31일 재고는 다음과 같이 구성되었다.\n\n- | 2018 | 2017\n완제품 | $988.1 | $1211.4\n가공 중 | 2628.2 | 2697.7\n원자재 및 용품 | 506.5 | 488.8\n총 (대략적인 교체 비용) | 4122.8 | 4397.9\n라이프 비용으로 증가(감소) | -11.0 (11.0) | 60.4\n재고 | $4111.8 | $4458.3\n\n2018년 12월 31일과 2017년 12월 31일에는 각각 15억 7천만 달러와 15억 6천만 달러의 총재고를 기록하였다. 주 7: 잠재적으로 신용위험을 받는 금융상품은 주로 매출채권과 이자부담 투자로 구성된다. 생명과학 제품의 도매 유통업자는 우리의 매출채권의 상당 부분을 차지하며, 일반적으로 담보는 필요하지 않습니다. 우리는 지속적인 신용평가 절차와 보험을 통해 이러한 집중과 관련된 위험을 완화하고자 한다. 우리의 현금의 상당 부분은 몇몇 주요 금융기관에 의해 보유되고 있다. 우리는 이들 기관과 함께 우리의 익스포저를 모니터링하며, 이들 기관 중 어느 기관도 의무를 이행하지 않을 것으로 기대하지 않는다. 주요 금융기관은 우리가 기업 부채 증권에 투자하는 가장 큰 부분을 대표한다. 문서화된 기업 리스크 관리 정책에 따라, 우리는 한 금융 기관 또는 기업 발행자의 신용 노출량을 모니터링한다. 우리는 위험관리 수단에 대한 거래상대방의 불이행 시 신용 관련 손실에 노출되어 있지만 높은 신용등급을 감안할 때 어떤 거래상대방도 의무를 이행하지 않을 것으로

In [64]:
with open('score_sample.json', 'w',encoding='utf-8') as f:
    json.dump(sample_list,f,ensure_ascii=False,indent=4)